In [1]:
import boto3
import pandas as pd
import io

s3 = boto3.client("s3")

def download_excel(bucket, key, sheet_name="Active Legal Contracts", column="Contract Number"):
    print(f"📥 Downloading Excel: s3://{bucket}/{key}")
    obj = s3.get_object(Bucket=bucket, Key=key)
    excel_data = obj['Body'].read()

    df = pd.read_excel(io.BytesIO(excel_data), sheet_name=sheet_name, engine='openpyxl')
    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in sheet '{sheet_name}'")

    return df

def list_s3_files_for_contract(bucket, contract_number, prefix_base="contract-docs/"):
    prefix = f"{prefix_base}{contract_number}/"
    files = []
    paginator = s3.get_paginator("list_objects_v2")

    for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
        for obj in page.get("Contents", []):
            files.append(obj["Key"])
    return files

def gather_contract_files(bucket, df, limit=1200):
    result = []
    file_count = 0

    # Clean columns
    df = df[['Contract Number', 'Account', 'Document Type']].dropna(subset=['Contract Number', 'Account'])

    # Convert to string and strip
    df['Contract Number'] = df['Contract Number'].astype(str).str.strip()
    df['Account'] = df['Account'].astype(str).str.strip()
    df['Document Type'] = df['Document Type'].astype(str).str.strip()

    grouped = df.groupby('Account')

    for client_account, group in grouped:
        group_contracts = group['Contract Number'].unique()

        temp_result = []

        for contract_number in group_contracts:
            doc_type = group[group['Contract Number'] == contract_number]['Document Type'].iloc[0]
            s3_files = list_s3_files_for_contract(bucket, contract_number)

            for key in s3_files:
                temp_result.append({
                    'contract_number': contract_number,
                    'client_account': client_account,
                    'S3_full_path': f"s3://{bucket}/{key}",
                    'file_name': key.split('/')[-1],
                    'doc_type': doc_type,
                    'S3_vectors': '--',
                    'OpenSearch': '--'
                })

        file_count += len(temp_result)

        result.extend(temp_result)

        if file_count >= limit:
            print(f"🛑 Reached file limit of {limit} after processing client: {client_account}")
            break

    return result

if __name__ == "__main__":
    bucket = "ml-legal-restricted"
    excel_key = "tabularData/Active Legal Contracts 7-10-2025 1-17-09 PM.xlsx"

    df = download_excel(bucket, excel_key)
    records = gather_contract_files(bucket, df, limit=1200)

    output_df = pd.DataFrame(records)
    output_df.to_csv("gathered_contract_files.csv", index=False)
    print("✅ CSV saved as 'gathered_contract_files.csv'")


📥 Downloading Excel: s3://ml-legal-restricted/tabularData/Active Legal Contracts 7-10-2025 1-17-09 PM.xlsx


/home/ubuntu/.local/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


🛑 Reached file limit of 1200 after processing client: Aetna Life Insurance Company
✅ CSV saved as 'gathered_contract_files.csv'


In [2]:
import pandas as pd

df = pd.read_csv("gathered_contract_files.csv")

total_files = len(df)

unique_clients = df['client_account'].nunique()

files_per_client = df.groupby('client_account')['file_name'].count().sort_values(ascending=False)

print(f"📁 Total Files Collected: {total_files}")
print(f"👤 Total Unique Clients: {unique_clients}")
print("\n📊 Files Per Client:\n")
print(files_per_client)

files_per_client.to_csv("files_per_client_summary.csv", header=['file_count'])
print("\n✅ Summary saved to 'files_per_client_summary.csv'")


📁 Total Files Collected: 1582
👤 Total Unique Clients: 120

📊 Files Per Client:

client_account
Aetna Life Insurance Company        666
Aerotek                              99
1199 SEIU National Benefit Funds     72
AArete LLC                           47
AAPC                                 45
                                   ... 
Acteva                                1
Advanced Systems Group                1
Advanced Document Systems             1
Adminisoft                            1
Advent International Corporation      1
Name: file_name, Length: 120, dtype: int64

✅ Summary saved to 'files_per_client_summary.csv'


In [2]:
import pandas as pd
import numpy as np

CSV1 = "contracts_files.csv"
CSV2 = "gathered_contract_files_valid.csv"
OUT_CSV = "gathered_contract_files_valid_new.csv"
TARGET_ROWS = 550
RNG_SEED = 42  # make selection reproducible

def build_new_valid_files(csv1=CSV1, csv2=CSV2, out_csv=OUT_CSV, target=TARGET_ROWS, seed=RNG_SEED):
    # Load
    df1 = pd.read_csv(csv1)
    df2 = pd.read_csv(csv2)

    # Basic checks
    if "s3_full_path" not in df1.columns:
        raise ValueError(f"'s3_full_path' not found in {csv1}")
    if "s3_full_path" not in df2.columns:
        raise ValueError(f"'s3_full_path' not found in {csv2}")
    if "account_name" not in df1.columns:
        raise ValueError(f"'account_name' not found in {csv1}")

    # Anti-join: keep rows from df1 whose s3_full_path is NOT present in df2
    df2_paths = df2["s3_full_path"].dropna().astype(str).unique()
    candidates = (
        df1[~df1["s3_full_path"].astype(str).isin(df2_paths)]
        .dropna(subset=["s3_full_path"])
        .drop_duplicates(subset=["s3_full_path"])
        .copy()
    )

    # Normalize/guard group key
    candidates["account_name"] = candidates["account_name"].fillna("(Unknown)")

    # Group and shuffle group order
    groups = list(candidates.groupby("account_name", as_index=False))
    rng = np.random.default_rng(seed)
    rng.shuffle(groups)

    # Select whole groups until we reach/exceed target
    selected_groups = []
    running_total = 0
    for acct, g in groups:
        selected_groups.append(g)
        running_total += len(g)
        if running_total >= target:
            break

    # If candidates < target, this will just be all candidates
    result = pd.concat(selected_groups, ignore_index=True) if selected_groups else candidates.head(0)
    result["s3_vectors"] = "--"
    result["opensearch"] = "--"

    # Save
    result.to_csv(out_csv, index=False)

    # Logging
    print(f"Candidates after anti-join: {len(candidates)} rows across {candidates['account_name'].nunique()} accounts.")
    print(f"Selected {len(selected_groups)} whole groups -> {len(result)} rows (target {target}).")
    print(f"Saved: {out_csv}")


build_new_valid_files()


Candidates after anti-join: 12170 rows across 1570 accounts.
Selected 56 whole groups -> 558 rows (target 550).
Saved: gathered_contract_files_valid_new.csv


In [1]:
#one time job for top15 clients + apply filters

"""
Contracts pipeline (final run, no CLI, no dry-run).

What it does
------------
- Load Excel from S3.
- Normalize fields:
    * Parent Contract -> clean string (no NaN, no trailing ".0")
    * Contract Title / Document Title -> single-line strings
    * Normalize SharePoint URL + extract file_name
- Core filters:
    * sharepoint_document_url non-empty
    * status_reason == "Fully-Executed/Complete" (case-insensitive)
    * account_type == "Client" (case-insensitive)
- Prioritization:
    * Keep ALL rows where solution_line == "Risk Adjustment"
    * For all other solution lines, keep rows whose account_name matches the top-15 clients
      (Horizon must look like Horizon BCBS/NJ; won't match "Horizon Systems")
- S3 file match:
    * Look under contract-docs/{contract_number}/ for exact file basename
    * If contract_number is numeric and < 8 digits, also try zero-padded to 8
- Outputs:
    * Writes `output_csv` with S3 paths + metadata
    * Writes `missing_files.csv` with rows where no S3 file match was found
- Returns:
    * (output_df, missing_df) as pandas DataFrames
"""

import os
import io
import re
from typing import Optional, Tuple
import boto3
import pandas as pd
from urllib.parse import urlparse, parse_qs, urlunparse, unquote

DEFAULT_SHEET = "Active Legal Contracts"

COLUMN_MAP = {
    "Contract Number": "contract_number",
    "Account": "account_name",
    "Document Type": "doc_type",
    "Status Reason": "status_reason",
    "Contract Title": "contract_title",
    "Contract Requester": "contract_requester",
    "Reviewing Attorney": "reviewing_attorney",
    "Created On": "created_on",
    "Document Effective Date": "document_effective_date",
    "Parent Contract": "parent_contract",
    "Solution Line": "solution_line",
    "Account Type": "account_type",
    "Document URL": "sharepoint_document_url",
    "Document Title": "document_title",
    "Related Product": "related_product",
}

CLIENT_REGEXES = [
    ("Anthem/Elevance", re.compile(r"\b(anthem|elevance)\b", re.I)),
    ("Aetna", re.compile(r"\baetna\b", re.I)),
    ("Humana", re.compile(r"\bhumana\b", re.I)),
    ("United", re.compile(r"\b(united\s*health( ?care)?|unitedhealth(group)?|uhc)\b", re.I)),
    ("Centene", re.compile(r"\bcentene\b", re.I)),
    ("HCSC", re.compile(r"\b(hcsc|health\s*care\s*service\s*corporation)\b", re.I)),
    ("BCBSTN", re.compile(r"\b(blue\s*cross\s*blue\s*shield\s*of\s*tennessee|bcbstn)\b", re.I)),
    ("Highmark", re.compile(r"\bhighmark\b", re.I)),
    ("UPMC", re.compile(r"\bupmc\b", re.I)),
    ("Horizon (NJ BCBS)", re.compile(r"\bhorizon\s*(blue|bcbs|cross|shield|new\s*jersey|nj)\b", re.I)),
    ("Excellus", re.compile(r"\bexcellus\b", re.I)),
    ("Wellmark", re.compile(r"\bwellmark\b", re.I)),
    ("BCBS Michigan", re.compile(r"\b(blue\s*cross\s*(and\s*)?blue\s*shield\s*of\s*michigan|bcbsm)\b", re.I)),
    ("Kaiser", re.compile(r"\bkaiser(\s+permanente)?\b", re.I)),
    ("BCBSLA", re.compile(r"\b(blue\s*cross\s*(and\s*)?blue\s*shield\s*of\s*louisiana|bcbsla)\b", re.I)),
]

def normalize_url(u: str) -> str:
    """Normalize URLs: strip, lowercase scheme/host, decode path, drop trailing slash on path."""
    if not isinstance(u, str):
        return ""
    u = u.strip()
    if not u:
        return ""
    try:
        p = urlparse(u)
        scheme = (p.scheme or "").lower()
        netloc = (p.netloc or "").lower()
        path = unquote(p.path or "")
        if path.endswith("/") and len(path) > 1:
            path = path.rstrip("/")
        return urlunparse((scheme, netloc, path, p.params, p.query, p.fragment))
    except Exception:
        return u

def filename_from_url(u: str) -> str:
    try:
        p = urlparse(u)
        qs = parse_qs(p.query)
        if "file" in qs and qs["file"]:
            return unquote(qs["file"][0])
        path = unquote(p.path or "")
        return os.path.basename(path)
    except Exception:
        return ""

def normalize_singleline(text) -> str:
    """Flatten multi-line/paragraph text into a single cleaned line."""
    if text is None or (isinstance(text, float) and pd.isna(text)):
        return ""
    s = str(text)
    s = s.replace("\r", " ").replace("\n", " ")
    s = re.sub(r"[\u2028\u2029]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def map_top_client(account_name: str) -> Tuple[Optional[str], Optional[str]]:
    """Return (canonical_label, matched_regex_pattern) or (None, None)."""
    if not isinstance(account_name, str) or not account_name.strip():
        return (None, None)
    for label, rx in CLIENT_REGEXES:
        if rx.search(account_name):
            return (label, rx.pattern)
    return (None, None)

def read_excel_from_s3(bucket: str, key: str, sheet_name: str = DEFAULT_SHEET,
                       s3_client: Optional[object] = None) -> pd.DataFrame:
    """Download Excel from S3 and return DataFrame."""
    s3 = s3_client or boto3.client("s3")
    try:
        excel_data = s3.get_object(Bucket=bucket, Key=key)["Body"].read()
    except Exception as e:
        raise RuntimeError(f"Failed to download Excel from s3://{bucket}/{key}: {e}")
    return pd.read_excel(io.BytesIO(excel_data), sheet_name=sheet_name, engine="openpyxl")


def base_transform(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Select/rename columns, coerce types, normalize titles/URLs, and apply core filters:
    - sharepoint_document_url non-empty
    - status_reason == Fully-Executed/Complete
    - account_type == Client
    """
    missing_cols = [col for col in COLUMN_MAP.keys() if col not in df_raw.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in Excel: {missing_cols}")

    df = df_raw[list(COLUMN_MAP.keys())].dropna(subset=["Contract Number"]).copy()

    df["Parent Contract"] = (
        df["Parent Contract"]
        .apply(lambda x: "" if pd.isna(x) else str(x).strip())
        .str.replace(r"\.0$", "", regex=True)
    )

    df["Contract Number"] = df["Contract Number"].astype(str).str.strip()

    df = df.rename(columns=COLUMN_MAP)

    if "contract_title" in df.columns:
        df["contract_title"] = df["contract_title"].apply(normalize_singleline)
    if "document_title" in df.columns:
        df["document_title"] = df["document_title"].apply(normalize_singleline)

    df["sharepoint_document_url"] = df["sharepoint_document_url"].fillna("").astype(str).apply(normalize_url)
    df["file_name"] = df["sharepoint_document_url"].apply(filename_from_url)

    df = df[df["sharepoint_document_url"].str.strip() != ""]

    df = df[df["status_reason"].fillna("").astype(str).str.strip().str.casefold()
            == "fully-executed/complete".casefold()]

    df = df[df["account_type"].fillna("").astype(str).str.strip().str.casefold() == "client"]

    return df.reset_index(drop=True)

def apply_prioritization(df: pd.DataFrame) -> pd.DataFrame:
    """Keep all Risk Adjustment; for others, keep only top-15 clients."""
    mapped = df["account_name"].fillna("").astype(str).apply(map_top_client)
    df = df.copy()
    df["matched_client"] = mapped.apply(lambda t: t[0])
    df["matched_pattern"] = mapped.apply(lambda t: t[1])

    ra_mask = df["solution_line"].fillna("").astype(str).str.strip().str.casefold() == "risk adjustment"
    keep_mask = ra_mask | (~ra_mask & df["matched_client"].notna())
    return df[keep_mask].reset_index(drop=True)

def find_matching_contract_file(bucket: str,
                                contract_number: str,
                                target_filename: str,
                                prefix_base: str = "contract-docs/",
                                s3_client: Optional[object] = None) -> Optional[str]:
    """
    Search under contract-docs/{contract_number}/ and return the *single* key
    that exactly matches the target file name. If not found, return None.
    """
    if not target_filename:
        return None

    s3 = s3_client or boto3.client("s3")
    prefix = f"{prefix_base}{contract_number}/"
    paginator = s3.get_paginator("list_objects_v2")

    try:
        for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
            for obj in page.get("Contents", []):
                key = obj.get("Key", "")
                if not key or key.endswith("/"):
                    continue
                if os.path.basename(key) == target_filename:
                    return key
    except Exception as e:
        print(f"⚠️  Error listing S3 for prefix {prefix}: {e}")

    return None

def process_contracts_with_metadata(bucket: str,
                                    excel_key: str,
                                    output_csv: str,
                                    sheet_name: str = DEFAULT_SHEET,
                                    prefix_base: str = "contract-docs/",
                                    s3_client: Optional[object] = None) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    End-to-end run:
      1) Read Excel from S3
      2) Transform + core filters
      3) Prioritize (Risk Adjustment OR top-15 clients)
      4) Match to S3 files
      5) Write outputs

    """

    df_raw = read_excel_from_s3(bucket, excel_key, sheet_name, s3_client=s3_client)
    df = base_transform(df_raw)
    df = apply_prioritization(df)

    output_rows = []
    not_found_rows = []

    for _, row in df.iterrows():
        contract_number = str(row.get("contract_number", "")).strip()
        file_name = str(row.get("file_name", "")).strip()

        key = find_matching_contract_file(bucket, contract_number, file_name,
                                          prefix_base=prefix_base, s3_client=s3_client)

        if not key and contract_number.isdigit() and len(contract_number) < 8:
            padded = contract_number.zfill(8)
            key = find_matching_contract_file(bucket, padded, file_name,
                                              prefix_base=prefix_base, s3_client=s3_client)

        if key:
            full_path = f"s3://{bucket}/{key}"
            entry = {"contract_number": contract_number, "s3_full_path": full_path}
            for csv_col in COLUMN_MAP.values():
                entry[csv_col] = row.get(csv_col, "")
            entry["matched_client"] = row.get("matched_client", "")
            entry["matched_pattern"] = row.get("matched_pattern", "")
            entry["file_name"] = file_name
            output_rows.append(entry)
        else:
            not_found_rows.append(
                {
                    "contract_number": contract_number,
                    "file_name": file_name,
                    "sharepoint_document_url": row.get("sharepoint_document_url", ""),
                    "account_name": row.get("account_name", ""),
                    "solution_line": row.get("solution_line", ""),
                }
            )

    output_df = pd.DataFrame(output_rows)
    missing_df = pd.DataFrame(not_found_rows)


    ordered_cols = (
        ["contract_number", "s3_full_path"]
        + list(COLUMN_MAP.values())
        + ["matched_client", "matched_pattern", "file_name"]
    )

    seen = set()
    ordered_cols = [c for c in ordered_cols if not (c in seen or seen.add(c))]

    if not output_df.empty:
        if "parent_contract" in output_df.columns:
            output_df["parent_contract"] = (
                output_df["parent_contract"]
                .apply(lambda x: "" if pd.isna(x) else str(x))
                .str.replace(r"\.0$", "", regex=True)
            )
        output_df = output_df.reindex(columns=ordered_cols, fill_value="")
        output_df.to_csv(output_csv, index=False)
    else:
        pd.DataFrame(columns=ordered_cols).to_csv(output_csv, index=False)

    missing_df.to_csv("missing_files.csv", index=False)

    return output_df, missing_df



out_df, missing_df = process_contracts_with_metadata(
    bucket="ml-legal-restricted",
    excel_key="tabularData/Active Legal Contracts 8-1-2025 10-54-06 AM.xlsx",
    output_csv="contracts_files_filtered.csv",
    sheet_name="Active Legal Contracts",
    prefix_base="contract-docs/",
)



/home/ubuntu/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# import argparse
# import os
# import re
# import pandas as pd
# from urllib.parse import urlparse, parse_qs, urlunparse, unquote
# from typing import Optional, Tuple

# # ---- Column mapping from Excel -> normalized names ----
# COLUMN_MAP = {
#     "Contract Number": "contract_number",
#     "Account": "account_name",
#     "Document Type": "doc_type",
#     "Status Reason": "status_reason",
#     "Contract Title": "contract_title",
#     "Contract Requester": "contract_requester",
#     "Reviewing Attorney": "reviewing_attorney",
#     "Created On": "created_on",
#     "Document Effective Date": "document_effective_date",
#     "Parent Contract": "parent_contract",
#     "Solution Line": "solution_line",
#     "Account Type": "account_type",
#     "Document URL": "sharepoint_document_url",
#     "Document Title": "document_title",
#     "Related Product": "related_product",
# }

# TOP15_CANONICAL = [
#     "Anthem/Elevance",
#     "Aetna",
#     "Humana",
#     "United",
#     "Centene",
#     "HCSC",
#     "BCBSTN",
#     "Highmark",
#     "UPMC",
#     "Horizon (NJ BCBS)",
#     "Excellus",
#     "Wellmark",
#     "BCBS Michigan",
#     "Kaiser",
#     "BCBSLA",
# ]

# # Carefully tuned regexes for the 15 clients
# CLIENT_REGEXES = [
#     ("Anthem/Elevance", re.compile(r"\b(anthem|elevance)\b", re.I)),
#     ("Aetna", re.compile(r"\baetna\b", re.I)),
#     ("Humana", re.compile(r"\bhumana\b", re.I)),
#     # Avoid generic "United": target UnitedHealthcare and common variants
#     ("United", re.compile(r"\b(united\s*health( ?care)?|unitedhealth(group)?|uhc)\b", re.I)),
#     ("Centene", re.compile(r"\bcentene\b", re.I)),
#     ("HCSC", re.compile(r"\b(hcsc|health\s*care\s*service\s*corporation)\b", re.I)),
#     ("BCBSTN", re.compile(r"\b(blue\s*cross\s*blue\s*shield\s*of\s*tennessee|bcbstn)\b", re.I)),
#     ("Highmark", re.compile(r"\bhighmark\b", re.I)),
#     ("UPMC", re.compile(r"\bupmc\b", re.I)),
#     # Horizon must look like Horizon BCBS/NJ — won't match "Horizon Systems"
#     ("Horizon (NJ BCBS)", re.compile(r"\bhorizon\s*(blue|bcbs|cross|shield|new\s*jersey|nj)\b", re.I)),
#     ("Excellus", re.compile(r"\bexcellus\b", re.I)),
#     ("Wellmark", re.compile(r"\bwellmark\b", re.I)),
#     ("BCBS Michigan", re.compile(r"\b(blue\s*cross\s*(and\s*)?blue\s*shield\s*of\s*michigan|bcbsm)\b", re.I)),
#     ("Kaiser", re.compile(r"\bkaiser(\s+permanente)?\b", re.I)),
#     ("BCBSLA", re.compile(r"\b(blue\s*cross\s*(and\s*)?blue\s*shield\s*of\s*louisiana|bcbsla)\b", re.I)),
# ]

# def normalize_url(u: str) -> str:
#     if not isinstance(u, str):
#         return ""
#     u = u.strip()
#     if not u:
#         return ""
#     try:
#         p = urlparse(u)
#         scheme = (p.scheme or "").lower()
#         netloc = (p.netloc or "").lower()
#         path = unquote(p.path or "")
#         if path.endswith("/") and len(path) > 1:
#             path = path.rstrip("/")
#         return urlunparse((scheme, netloc, path, p.params, p.query, p.fragment))
#     except Exception:
#         return u

# def filename_from_url(u: str) -> str:
#     try:
#         p = urlparse(u)
#         qs = parse_qs(p.query)
#         if "file" in qs and qs["file"]:
#             return unquote(qs["file"][0])
#         path = unquote(p.path or "")
#         return os.path.basename(path)
#     except Exception:
#         return ""

# def map_top_client(account_name: str) -> Tuple[Optional[str], Optional[str]]:
#     """Return (canonical_label, matched_regex_pattern) or (None, None)."""
#     if not isinstance(account_name, str) or not account_name.strip():
#         return (None, None)
#     for label, rx in CLIENT_REGEXES:
#         if rx.search(account_name):
#             return (label, rx.pattern)
#     return (None, None)

# def is_top_client(account_name: str) -> bool:
#     label, _ = map_top_client(account_name)
#     return label is not None

# def load_and_filter_excel(path: str, sheet_name: str) -> pd.DataFrame:
#     # Read Excel
#     df = pd.read_excel(path, sheet_name=sheet_name, engine="openpyxl")

#     # Validate required columns
#     missing = [c for c in COLUMN_MAP.keys() if c not in df.columns]
#     if missing:
#         raise ValueError(f"Missing columns in Excel: {missing}")

#     # Select & coerce
#     df = df[list(COLUMN_MAP.keys())].dropna(subset=["Contract Number"]).copy()

#     # Parent Contract -> clean string (no NaN, trim, drop trailing ".0")
#     df["Parent Contract"] = (
#         df["Parent Contract"]
#         .apply(lambda x: "" if pd.isna(x) else str(x).strip())
#         .str.replace(r"\.0$", "", regex=True)
#     )

#     # Contract Number as string
#     df["Contract Number"] = df["Contract Number"].astype(str).str.strip()

#     # Rename to normalized names
#     df = df.rename(columns=COLUMN_MAP)

#     # Normalize/derive URL + filename
#     df["sharepoint_document_url"] = df["sharepoint_document_url"].fillna("").astype(str).apply(normalize_url)
#     df["file_name"] = df["sharepoint_document_url"].apply(filename_from_url)

#     # ---- Core filters (requested) ----
#     # 1) non-empty URL
#     df = df[df["sharepoint_document_url"].str.strip() != ""]
#     # 2) Status Reason exactly Fully-Executed/Complete (case-insensitive, trimmed)
#     df = df[df["status_reason"].fillna("").astype(str).str.strip().str.casefold()
#             == "fully-executed/complete".casefold()]
#     # 3) Account Type == Client
#     df = df[df["account_type"].fillna("").astype(str).str.strip().str.casefold() == "client"]

#     return df

# def prioritize_rows(df: pd.DataFrame) -> pd.DataFrame:
#     # Add matched client info
#     mapped = df["account_name"].fillna("").astype(str).apply(map_top_client)
#     df = df.copy()
#     df["matched_client"] = mapped.apply(lambda t: t[0])
#     df["matched_pattern"] = mapped.apply(lambda t: t[1])

#     # Always keep Solution Line == "Risk Adjustment"
#     ra_mask = df["solution_line"].fillna("").astype(str).str.strip().str.casefold() == "risk adjustment"

#     # For other solution lines, keep only top-15 clients
#     keep_mask = ra_mask | (~ra_mask & df["matched_client"].notna())

#     return df[keep_mask].reset_index(drop=True)

# def process_contracts_with_metadata():
#     EXCEL_KEY = "Active Legal Contracts 8-1-2025 10-54-06 AM.xlsx"
#     sheet_name="Active Legal Contracts"

#     df = load_and_filter_excel(EXCEL_KEY, sheet_name)
#     df_out = prioritize_rows(df)

#     # Choose a helpful column order (only include those that exist)
#     desired_cols = [
#         "contract_number", "account_name", "matched_client", "solution_line",
#         "status_reason", "account_type",
#         "contract_title", "document_title", "doc_type", "related_product",
#         "created_on", "document_effective_date", "parent_contract",
#         "sharepoint_document_url", "file_name",
#     ]
#     cols = [c for c in desired_cols if c in df_out.columns]
#     if cols:
#         df_out = df_out[cols]

#     df_out.to_csv('contracts_files_dry_run.csv', index=False)

#     print(f"\n✅ Wrote {len(df_out)} rows to CSV")
#     print("\n📊 Counts by matched client label:")
#     print(df_out["matched_client"].value_counts(dropna=False).to_string())
#     print("\n📊 Counts by solution line (kept):")
#     print(df_out["solution_line"].value_counts(dropna=False).to_string())

# if __name__ == "__main__":
#     process_contracts_with_metadata()


/home/ubuntu/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)



✅ Wrote 3505 rows to CSV

📊 Counts by matched client label:
matched_client
Anthem/Elevance      1301
United                370
Aetna                 357
Kaiser                276
Centene               242
Highmark              210
Humana                194
None                  114
HCSC                   87
UPMC                   81
Horizon (NJ BCBS)      69
BCBS Michigan          58
Excellus               44
BCBSTN                 42
BCBSLA                 32
Wellmark               28

📊 Counts by solution line (kept):
solution_line
Population Health Management    2080
Payment Accuracy                 701
Quality Improvement              208
Risk Adjustment                  200
Performance Analytics            114
Edifecs                           80
Payment Integrity                 62
Record Import/Clean Up            60
